In [2]:
import pandas as pd
import os

# opening relevant files
icd_codes = pd.read_csv('/Users/minookim/Desktop/LCICM/mimic4_ehr/diagnoses_icd.csv')
icu_stays = pd.read_csv('/Users/minookim/Desktop/LCICM/mimic4_ehr/icustays.csv')
lab_events = pd.read_csv('/Users/minookim/Desktop/LCICM/mimic4_ehr/labevents.csv')
lab_events_id = pd.read_csv('/Users/minookim/Desktop/LCICM/mimic4_ehr/d_labitems.csv')
lab_events = pd.merge(lab_events, lab_events_id, on='itemid', how='left')
print ("File reading complete")

File reading complete


In [3]:
icu_stays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
73177,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
73178,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
73179,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [4]:
# extract patient IDs with 12-lead ECGs
ecg_ids = [f.name for f in os.scandir('/Users/minookim/Desktop/LCICM/ecg_extraction/mimic-iv-ecg/files/')]
ecg_ids.remove(".DS_Store")
# take out p and convert to number
ecg_ids = [id[1:] for id in ecg_ids]
ecg_ids = [int(i) for i in ecg_ids]
print(len(ecg_ids)) # evaluates to 93
print(ecg_ids)

92
[10005866, 10009035, 10018328, 10019003, 10007795, 10039831, 10029291, 10002428, 10025463, 10022017, 10020740, 10021666, 10015860, 10020187, 10021312, 10021118, 10005348, 10001725, 10027445, 10012552, 10010867, 10004235, 10019917, 10014078, 10010471, 10021938, 10017492, 10018081, 10007058, 10015931, 10023239, 10011398, 10038081, 10016742, 10004422, 10026255, 10038992, 10002495, 10001217, 10020306, 10040025, 10005817, 10015272, 10014354, 10012853, 10037928, 10038933, 10037975, 10022880, 10022041, 10020944, 10014729, 10004720, 10035631, 10016810, 10031404, 10004457, 10000032, 10002930, 10016150, 10003046, 10008454, 10035185, 10036156, 10007818, 10019777, 10013049, 10019385, 10019172, 10037861, 10032725, 10018423, 10027602, 10039997, 10020640, 10018845, 10003400, 10024043, 10031757, 10039708, 10008287, 10006580, 10023117, 10006053, 10029484, 10025612, 10009628, 10023771, 10004733, 10021487, 10038999, 10009049]


In [5]:
# total ICU population
print(len(icu_stays))

# ICU population w/ 24 hour data
icu_stays = icu_stays[icu_stays['los']>=1]
icu_stays=icu_stays.drop_duplicates('hadm_id')
icu_ids = list(icu_stays.hadm_id)
print(len(icu_ids)) 
print(icu_stays['subject_id'].nunique()) # e: 53034, p: 42264

73181
53034
42264


In [6]:
# filter every file with confirmed ICU stay
icd_codes = icd_codes[icd_codes['hadm_id'].isin(icu_ids)]
lab_events = lab_events[lab_events['hadm_id'].isin(icu_ids)]

In [7]:
# filter for HFpEF w/ ICD codes
hfpef_icd9 = icd_codes[icd_codes['icd_code'].str.startswith('4283')]
hfpef_icd10 = icd_codes[icd_codes['icd_code'].str.startswith('I503')]
hfpef_ids = pd.concat([hfpef_icd9, hfpef_icd10])
hfpef_ids = hfpef_ids.drop_duplicates('hadm_id')

hfpef_ids = hfpef_ids['hadm_id']
hfpef_ids = pd.merge(hfpef_ids, icu_stays, on='hadm_id', how="left")
# for encounter 
print(len(hfpef_ids))
# for patient 
print(hfpef_ids['subject_id'].nunique())

6116
4786


In [8]:
probnp_total = lab_events[(lab_events['itemid'] == 50963)]
probnp_total = probnp_total.dropna(subset=['valuenum'])
probnp_total = probnp_total.drop_duplicates('hadm_id')
probnp_total_ids  = list(probnp_total.hadm_id)
print(len(probnp_total_ids))

# filter for proBNP over and under 200
probnp_positive = probnp_total[probnp_total['valuenum'] > 200]
probnp_positive_ids = list(probnp_positive.hadm_id)
print(len(probnp_positive_ids))

# < 200
probnp_negative = probnp_total[probnp_total['valuenum'] <= 200]
probnp_negative_ids = list(probnp_negative.hadm_id)
print(len(probnp_negative_ids))

5542
5216
326


In [20]:
from datetime import datetime, timedelta

# HFpEF: proBNP + ICD codes
hfpef_icd_positive = hfpef_ids.drop('subject_id', axis=1)
hfpef_icd_positive = pd.merge(hfpef_icd_positive, probnp_positive, on='hadm_id', how="left")
hfpef_icd_positive = hfpef_icd_positive.dropna(subset=['valuenum'])

print("before time filter")
print(len(hfpef_icd_positive))
print(hfpef_icd_positive['subject_id'].nunique())


### IGNORE
# testing for suitable time interval
print("testing for x time before lab measurement")
test_hfpef_icd_positive = hfpef_icd_positive[["charttime", "intime", "outtime"]]
# convert to datetime object
def parse_date(str):
    return datetime.strptime(str, '%Y-%m-%d %H:%M:%S')
def get_date(datetime):
    return datetime.day

test_hfpef_icd_positive_ct = test_hfpef_icd_positive["charttime"].apply(parse_date)
test_hfpef_icd_positive_it = test_hfpef_icd_positive["intime"].apply(parse_date)
time_diff = pd.DataFrame(test_hfpef_icd_positive_ct-test_hfpef_icd_positive_it, columns=["diff"])
print(len(time_diff[time_diff["diff"].dt.total_seconds()/60/60 >= -1]))
print(len(time_diff[time_diff["diff"].dt.total_seconds()/60/60 >= -3]))
print(len(time_diff[time_diff["diff"].dt.total_seconds()/60/60 >= -6]))
print(len(time_diff[time_diff["diff"].dt.total_seconds()/60/60 >= -12]))
print(len(time_diff[time_diff["diff"].dt.total_seconds()/60/60/24 >= -1]))
print(len(time_diff[time_diff["diff"].dt.total_seconds()/60/60/24 >= -2]))
print(len(time_diff[time_diff["diff"].dt.total_seconds()/60/60/24 >= -3]))
print(len(time_diff[time_diff["diff"].dt.total_seconds()/60/60/24 >= -4]))
print("testing for x time before lab measurement")
### IGNORE 

# set intime cutoff to 30 days before measurement

print("after time filter")
# filter additionally based on time
# convert to datetime objects
hfpef_icd_positive["intime"] = hfpef_icd_positive["intime"].apply(parse_date)
hfpef_icd_positive["charttime"] = hfpef_icd_positive["charttime"].apply(parse_date)
hfpef_icd_positive = hfpef_icd_positive[hfpef_icd_positive['charttime'] >= hfpef_icd_positive['intime'] - timedelta(days=30)]
hfpef_icd_positive = hfpef_icd_positive[hfpef_icd_positive['charttime'] <= hfpef_icd_positive['outtime']]
print(len(hfpef_icd_positive))
print(hfpef_icd_positive['subject_id'].nunique()) # e: 1224, p: 1157


# HFpEF: proBNP + ICD codes (false sample)
hfpef_icd_negative = hfpef_ids.drop('subject_id', axis=1)
hfpef_icd_negative = pd.merge(hfpef_icd_negative, probnp_negative, on='hadm_id', how="left")
hfpef_icd_negative = hfpef_icd_negative.dropna(subset=['valuenum'])

print("before time filter")
print(len(hfpef_icd_negative))
print(hfpef_icd_negative['subject_id'].nunique())

# filter additionally based on time
# convert to datetime objects
hfpef_icd_negative["intime"] = hfpef_icd_negative["intime"].apply(parse_date)
hfpef_icd_negative["charttime"] = hfpef_icd_negative["charttime"].apply(parse_date)
hfpef_icd_negative = hfpef_icd_negative[hfpef_icd_negative['charttime'] >= hfpef_icd_negative['intime'] - timedelta(days=30)]
hfpef_icd_negative = hfpef_icd_negative[hfpef_icd_negative['charttime'] <= hfpef_icd_negative['outtime']]
print("after time filter")
print(len(hfpef_icd_negative))
print(hfpef_icd_negative['subject_id'].nunique()) # e: 44, p: 44

# HFpEF: proBNP + ICD codes + ECG
hfpef_icd_ecg = hfpef_icd_positive[hfpef_icd_positive['subject_id'].isin(ecg_ids)]
print(len (hfpef_icd_ecg)) 
print(hfpef_icd_ecg['subject_id'].nunique()) # e: 4 p: 3

before time filter
1501
1401
testing for x time before lab measurement
788
828
896
960
1154
1240
1280
1308
testing for x time before lab measurement
after time filter
1224
1157
before time filter
54
54
after time filter
44
44
4
3


In [10]:
# filter for HFrEF w/ ICD codes
hfref_icd9 = icd_codes[icd_codes['icd_code'].str.startswith('4282')]
hfref_icd10 = icd_codes[icd_codes['icd_code'].str.startswith('I502')]
hfref_ids = pd.concat([hfref_icd9, hfref_icd10])
hfref_ids = hfref_ids.drop_duplicates('hadm_id')
# for encounter 
print(len(hfref_ids))
# for patient 
print(hfref_ids['subject_id'].nunique())
hfref_ids = hfref_ids['hadm_id']
hfref_ids = pd.merge(hfref_ids, icu_stays, on='hadm_id', how="left")

5924
4733


In [23]:
# HFrEF: proBNP + ICD codes
hfref_icd_positive = hfref_ids.drop('subject_id', axis=1)
hfref_icd_positive = pd.merge(hfref_icd_positive, probnp_positive, on='hadm_id', how="left")
hfref_icd_positive = hfref_icd_positive.dropna(subset=['valuenum'])
print("before time filter")
print(len(hfref_icd_positive))
print(hfref_icd_positive['subject_id'].nunique())

# filter additionally based on time
hfref_icd_positive["intime"] = hfref_icd_positive["intime"].apply(parse_date)
hfref_icd_positive["charttime"] = hfref_icd_positive["charttime"].apply(parse_date)
hfref_icd_positive = hfref_icd_positive[hfref_icd_positive['charttime'] >= hfref_icd_positive['intime'] - timedelta(days=30)]
hfref_icd_positive = hfref_icd_positive[hfref_icd_positive['charttime'] <= hfref_icd_positive['outtime']]
print("after time filter")
print(len(hfref_icd_positive))
print(hfref_icd_positive['subject_id'].nunique()) # e: 504, p: 494


# HFrEF: proBNP + ICD codes (false sample)
hfref_icd_negative = hfref_ids.drop('subject_id', axis=1)
hfref_icd_negative = pd.merge(hfref_icd_negative, probnp_negative, on='hadm_id', how="left")
hfref_icd_negative = hfref_icd_negative.dropna(subset=['valuenum'])
print("before time filter")
print(len (hfref_icd_negative)) 
print(hfref_icd_negative['subject_id'].nunique())

# filter additionally based on time
hfref_icd_negative["intime"] = hfref_icd_negative["intime"].apply(parse_date)
hfref_icd_negative["charttime"] = hfref_icd_negative["charttime"].apply(parse_date)
hfref_icd_negative = hfref_icd_negative[hfref_icd_negative['charttime'] >= hfref_icd_negative['intime'] - timedelta(days =30)]
hfref_icd_negative = hfref_icd_negative[hfref_icd_negative['charttime'] <= hfref_icd_negative['outtime']]
print("after time filter")
print(len(hfref_icd_negative))
print(hfref_icd_negative['subject_id'].nunique()) # e: 15, p: 15



# HFrEF: proBNP + ICD codes + ECG
hfref_icd_ecg = hfref_icd_positive[hfref_icd_positive['subject_id'].isin(ecg_ids)]
print(len (hfref_icd_ecg)) 
print(hfref_icd_ecg['subject_id'].nunique()) # e: 0 p: 0

before time filter
1321
1229
after time filter
1094
1033
before time filter
15
15
after time filter
13
13
1
1


In [26]:
# all ICD patients
chf_codes = pd.concat([hfpef_ids, hfref_ids])
chf_codes = chf_codes.drop_duplicates('hadm_id')
chf_codes_ids = list(chf_codes.hadm_id)
print(len(chf_codes)) 
print(chf_codes['subject_id'].nunique()) # e: 12015 p: 9232

icu_notin_icd = icu_stays[~icu_stays['hadm_id'].isin(chf_codes_ids)]
print(len(icu_notin_icd)) # evaluates to 41019

# no ICD but has proBNP
icu_has_probnp = icu_notin_icd[icu_notin_icd['hadm_id'].isin(probnp_total_ids)]
print(len(icu_has_probnp)) 
print(icu_has_probnp['subject_id'].nunique()) # e: 2662, p: 2591

# no ICD but proBNP >200
icu_probnp_positive = icu_has_probnp.drop('subject_id', axis=1)
icu_probnp_positive = pd.merge(icu_probnp_positive, probnp_positive, on='hadm_id', how="left")
icu_probnp_positive = icu_probnp_positive.dropna(subset=['valuenum'])
# filter additionally based on time
icu_probnp_positive = icu_probnp_positive[icu_probnp_positive['charttime'] >= icu_probnp_positive['intime']]
icu_probnp_positive = icu_probnp_positive[icu_probnp_positive['charttime'] <= icu_probnp_positive['outtime']]
print(len(icu_probnp_positive))
print(icu_probnp_positive['subject_id'].nunique()) # e: 1041, p: 1031

# no ICD but proBNP > 200 + ECGs
icu_probnp_positive_ecg = icu_probnp_positive[icu_probnp_positive['subject_id'].isin(ecg_ids)]
print(len(icu_probnp_positive_ecg)) 
print(icu_probnp_positive_ecg['subject_id'].nunique()) # e: 3, p: 3



# no ICD but proBNP >200
icu_probnp_negative = icu_has_probnp.drop('subject_id', axis=1)
icu_probnp_negative = pd.merge(icu_probnp_negative, probnp_negative, on='hadm_id', how="left")
icu_probnp_negative = icu_probnp_negative.dropna(subset=['valuenum'])
# filter additionally based on time
icu_probnp_negative = icu_probnp_negative[icu_probnp_negative['charttime'] >= icu_probnp_negative['intime']]
icu_probnp_negative = icu_probnp_negative[icu_probnp_negative['charttime'] <= icu_probnp_negative['outtime']]
print(len(icu_probnp_negative))
print(icu_probnp_negative['subject_id'].nunique()) # e: 119, p: 119

# no ICD but proBNP < 200 + ECGs
icu_probnp_negative_ecg = icu_probnp_negative[icu_probnp_negative['subject_id'].isin(ecg_ids)]
print(len(icu_probnp_negative_ecg)) 
print(icu_probnp_negative_ecg['subject_id'].nunique()) # e: 1, p: 1

12015
9232
41019
2662
2591
1041
1031
3
3
119
119
1
1


In [25]:
# no ICD AND no proBNP
icu_no_icd_no_probnp = icu_notin_icd[~icu_notin_icd['hadm_id'].isin(probnp_total_ids)]
print(len(icu_no_icd_no_probnp)) 
print(icu_no_icd_no_probnp['subject_id'].nunique()) # e: 38357, p: 32690

icu_no_icd_no_probnp2 = icu_notin_icd[icu_notin_icd['hadm_id'].isin(probnp_total_ids)]
print(len(icu_no_icd_no_probnp2)) 


# no ICD AND no proBNP + ECGs
icu_no_icd_no_probnp_ecg = icu_no_icd_no_probnp[icu_no_icd_no_probnp['subject_id'].isin(ecg_ids)]
print(len(icu_no_icd_no_probnp_ecg)) 
print(icu_no_icd_no_probnp_ecg['subject_id'].nunique()) # e: 76, p: 67

38357
32690
2662
76
67


In [14]:
# export csvs for feature extraction

hfpef_icd_ecg.to_csv("hfpef_icd_ecg.csv", sep=',', index=False, encoding='utf-8')
hfref_icd_ecg.to_csv("hfref_icd_ecg.csv", sep=',', index=False, encoding='utf-8')
icu_probnp_positive_ecg.to_csv("icu_probnp_positive_ecg.csv", sep=',', index=False, encoding='utf-8')
icu_probnp_negative_ecg.to_csv("icu_probnp_negative_ecg.csv", sep=',', index=False, encoding='utf-8')
icu_no_icd_no_probnp_ecg.to_csv("icu_no_icd_no_probnp_ecg.csv", sep=',', index=False, encoding='utf-8')